In [19]:
import numpy as np
import pandas as pd
from bitmex import bitmex
import math
import os.path
import time
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook

In [20]:
key = 'w0Vc7Xs9AKChVqN_jHXWosp3'
secret = 'EaO2tP5muocUl6-xFMKXKlwDFRrx1DYNtOxrsyjkv1KnNFl_'

In [21]:
symbol = "XBTUSD"
kline_size = "1h"

In [22]:
bin_length = {"1m": 1, "15m": 15, "30m": 30, "1h": 60}
batch_size = 750
client = bitmex(test=False, api_key=key, api_secret=secret)

In [23]:
def update(symbol, kline_size, data):
    if len(data)!=0:  old = parser.parse(data["timestamp"].iloc[-1])
    old = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    new = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

In [24]:
def fetch(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data1 = pd.read_csv(filename)
    else: data1 = pd.DataFrame()
    oldest_point, newest_point = update(symbol, kline_size, data1)
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/bin_length[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds>0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * bin_length[kline_size]))
            data = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data1 = data1.append(temp_df)
    data1.set_index('timestamp', inplace=True)
    if save and rounds > 0: data1.to_csv(filename)
    print('Done')
    return data1

In [25]:
data_final = fetch(symbol, kline_size, save=True)

<ipython-input-24-f8449d5a3ce1>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for round_num in tqdm_notebook(range(rounds)):


  0%|          | 0/78 [00:00<?, ?it/s]

KeyboardInterrupt: 